In [152]:
import pandas as pd
import numpy as np
import geopandas as gpd

pd.set_option('display.max_rows', 50)
pd.set_option('display.max_columns', 50)

In [234]:
hogares = pd.read_csv('Encuesta de Movilidad 2019/BD EODH2019 FINAL v14022020/Archivos CSV/HogaresEODH2019.csv', sep=';')
personas = pd.read_csv('Encuesta de Movilidad 2019/BD EODH2019 FINAL v14022020/Archivos CSV/PersonasEODH2019.csv',sep=';')
etapas = pd.read_csv('Encuesta de Movilidad 2019/BD EODH2019 FINAL v14022020/Archivos CSV/EtapasEODH2019.csv',sep=';')
viajes = pd.read_csv('Encuesta de Movilidad 2019/BD EODH2019 FINAL v14022020/Archivos CSV/ViajesEODH2019.csv')

/usr/local/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3063: DtypeWarning: Columns (12,13) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/usr/local/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3063: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [5]:
hogares_persona = pd.merge(hogares, personas, left_on='Id_Hogar', right_on='id_hogar')

In [144]:
ideca = gpd.read_file('salud/salud_ideca.geojson')
hospitales = ideca[ideca.NGeClasifi=='SALUD']
hospitales = hospitales[(hospitales.NGeNombre.str.contains('Hospital'))
                         |(hospitales.NGeNombre.str.contains('Clínica'))
                         |(hospitales.NGeNombre.str.contains('Clínica'))
                         |(hospitales.NGeNombre.str.contains('Fundación'))
                         |(hospitales.NGeNombre.str.contains('Fundacion'))].reset_index(drop=True)
hospitales['ZAT'] = 0
zats = gpd.read_file('ZONAS/ZAT.shp')

for hindex, hrow in hospitales.iterrows():
    for zindex, zrow in zats.iterrows():
        if(hrow.geometry.within(zrow.geometry)):
            hospitales.loc[hindex,'ZAT'] = zrow.ZAT
            
hospitales.ZAT = hospitales.ZAT.astype(int)            
hospitales['longitud'] = hospitales.geometry.x
hospitales['latitud'] = hospitales.geometry.y

hospitales['departamento'] = 'Bogota, D.C.'
hospitales['municipio'] = 'Bogota, D.C.'
hospitales['cod_prestador'] = '1100100032'
hospitales['prestador'] = hospitales.NGeNombre
hospitales['cod_habilitacion'] = '1100100032'
hospitales['nombre_sede'] = hospitales.NGeNombre
hospitales['cod_sede'] = hospitales.NGeIdentif
hospitales['gerente'] = ''
hospitales['zona'] = 'Urbana'

hospitales = hospitales[['departamento', 'municipio', 'cod_prestador', 'prestador', 'cod_habilitacion', 
            'cod_sede', 'nombre_sede', 'gerente', 'zona', 'ZAT', 'latitud', 'longitud']]

hospitales.to_csv('data_outputs/hospitales.csv')

In [257]:
personas_salud = personas[personas['p7_id_actividad_economica'].isin([17])
                         ][['id_hogar', 'id_persona', 'p6_id_ocupacion',
                            'p7_id_actividad_economica','p4_edad','Sexo', 'f_exp']]

In [288]:
viajes_persona = pd.merge(personas_salud, viajes[['id_hogar', 'id_persona', 'id_viaje','fecha', 'zat_origen', 
                                                  'lugar_origen', 'p17_Id_motivo_viaje', 'hora_inicio_viaje', 
                                                  'p28_lugar_destino', 'zat_destino', 'p31_hora_llegada', 
                                                  'modo_principal']], 
                          on=['id_hogar', 'id_persona'])

viajes_persona = viajes_persona[~viajes_persona.zat_origen.isna()]
viajes_persona = viajes_persona[~viajes_persona.zat_destino.isna()]

viajes_persona.zat_destino = viajes_persona.zat_destino.astype(int)
viajes_persona.zat_origen = viajes_persona.zat_origen.astype(int)

In [269]:
viajes_persona[viajes_persona.modo_principal=='TransMilenio'].f_exp.sum()/viajes[viajes.modo_principal=='TransMilenio'].f_exp.sum()

0.01862755592969233